# Just a sandboy to check various things interactively in a notebook

In [1]:
import pandas as pd
import os
import glob
import pandas as pd
from generate_overview import generate_summary
from utils.plotter import plot_donations_by_date, plot_data_points_by_category

# Get list of all CSV files in the overview directory
overview_files = glob.glob('./data/overview/overview_*.csv')
# Find most recent file based on modification time
latest_file = max(overview_files, key=os.path.getmtime)
# Read the most recent CSV file
df = pd.read_csv(latest_file)

# Filter entries before start of data collection -> must be refined. (17th offical launch)
df = df[df['start_time'] >= '2025-02-17']

# Filter out 99 (test case)
# Filter out explicit strings from Q2_age while keeping numeric values or empty values (removes testcases)
string_mask = (df['Q2_age'].astype(str).str.match(r'^[A-Za-z]+$')) & (df['Q2_age'].astype(str) != 'nan')
df = df[~string_mask]
df = df[df['Q2_age'] != '99']
n_started = df.shape[0]


n_finished = df['completed'].eq(True).sum()
df = df[pd.to_numeric(df['Q2_age'], errors='coerce').notna()]



/Users/lion.wedel/gitProjects/datadonation-wi-monitoring/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
len(df)

831

In [3]:
df_completed = df[df["completed"] == True]
len(df_completed)


830

In [4]:
df_consent = df_completed[df_completed["AngeseheneVideos_consent"] == True]
len(df_consent)


777

In [5]:
# First convert the end_time column to datetime
df_consent['end_time'] = pd.to_datetime(df_consent['end_time'])

# Now you can use the dt accessor
df_today = df_consent[df_consent["end_time"].dt.date == pd.to_datetime("2025-02-27").date()]

/var/folders/qn/fxb4jn2d09gbf70_jjtqy3dc0000gr/T/ipykernel_3884/706759970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consent['end_time'] = pd.to_datetime(df_consent['end_time'])


In [6]:
for i in df_today.columns:
    print(i)


participant_id
start_time
end_time
completed
extra_data
current_step
Q1_gender
Q2_age
Q3_education
Q4_location
Q5_first_vote
Q6_second_vote
Q7_polInt-0
Q7_polInt-1
time_submitted
handled
AngeseheneVideos_consent
AngeseheneVideos_status
AngeseheneVideos_n_datapoints
Likes_consent
Likes_status
Likes_n_datapoints
Suchen_consent
Suchen_status
Suchen_n_datapoints
Shares_consent
Shares_status
Shares_n_datapoints
Posts_consent
Posts_status
Posts_n_datapoints
Kommentare_consent
Kommentare_status
Kommentare_n_datapoints
FolgendeAccounts_consent
FolgendeAccounts_status
FolgendeAccounts_n_datapoints
GefolgteAccounts_consent
GefolgteAccounts_status
GefolgteAccounts_n_datapoints
BlockierteAccounts_consent
BlockierteAccounts_status
BlockierteAccounts_n_datapoints


In [7]:
df_check = df_today.loc[:,['participant_id',"AngeseheneVideos_status",'AngeseheneVideos_n_datapoints',
                'Likes_status','Likes_n_datapoints',
                'Suchen_status','Suchen_n_datapoints',
                'Shares_status','Shares_n_datapoints',
                'Posts_status','Posts_n_datapoints',
                'Kommentare_status','Kommentare_n_datapoints',
                'FolgendeAccounts_status','FolgendeAccounts_n_datapoints',
                'GefolgteAccounts_status','GefolgteAccounts_n_datapoints',
                'BlockierteAccounts_status','BlockierteAccounts_n_datapoints']]


In [8]:
df_check[df_check["AngeseheneVideos_n_datapoints"] == 0].describe()

,AngeseheneVideos_n_datapoints,Likes_n_datapoints,Suchen_n_datapoints,Shares_n_datapoints,Posts_n_datapoints,Kommentare_n_datapoints,FolgendeAccounts_n_datapoints,GefolgteAccounts_n_datapoints,BlockierteAccounts_n_datapoints
count,32.0,32.0,32.0,32.0,32.000000,32.000000,32.0,32.0,32.000000
mean,0.0,0.0,0.0,0.0,3.468750,196.312500,0.0,0.0,6.843750
std,0.0,0.0,0.0,0.0,12.247408,454.398961,0.0,0.0,30.247484
min,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
25%,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
50%,0.0,0.0,0.0,0.0,0.000000,9.500000,0.0,0.0,0.000000
75%,0.0,0.0,0.0,0.0,0.000000,56.750000,0.0,0.0,4.000000
max,0.0,0.0,0.0,0.0,60.000000,1646.000000,0.0,0.0,172.000000


In [9]:
df_failed = df_today[df_today["AngeseheneVideos_n_datapoints"] == 0]

In [10]:
df_today[df_today["AngeseheneVideos_n_datapoints"] > 0]

,participant_id,start_time,end_time,completed,extra_data,current_step,Q1_gender,Q2_age,Q3_education,Q4_location,...,Kommentare_n_datapoints,FolgendeAccounts_consent,FolgendeAccounts_status,FolgendeAccounts_n_datapoints,GefolgteAccounts_consent,GefolgteAccounts_status,GefolgteAccounts_n_datapoints,BlockierteAccounts_consent,BlockierteAccounts_status,BlockierteAccounts_n_datapoints
8,04taxYxElIagIqxkveekikY6,2025-02-26T20:16:13.972234+01:00,2025-02-27 10:10:59.809984+01:00,True,{'url_param': {}},3,0.0,24,7.0,9.0,...,0.0,True,success,1.0,True,success,110.0,True,nothing extracted,0.0
19,0C5qcf2jvLjLSja7IwwXvigM,2025-02-27T05:31:55.378595+01:00,2025-02-27 05:33:00.596036+01:00,True,{'url_param': {}},3,1.0,21,6.0,1.0,...,104.0,True,success,44.0,True,success,88.0,True,success,0.0
48,0QYh4UT6o6rhqa1qLOVnLneO,2025-02-26T15:06:14.995030+01:00,2025-02-27 00:04:50.469791+01:00,True,{'url_param': {}},3,0.0,28,8.0,2.0,...,491.0,True,success,146.0,True,success,233.0,True,success,29.0
50,0RmdGjtSv9obO5q35IJHzAKi,2025-02-27T13:25:45.585415+01:00,2025-02-27 13:27:00.188120+01:00,True,{'url_param': {}},3,0.0,21,7.0,13.0,...,3.0,True,success,5.0,True,success,26.0,True,nothing extracted,0.0
74,0bNdvLFemcezt5yKQbHU6GvU,2025-02-27T07:22:41.774838+01:00,2025-02-27 07:34:45.185800+01:00,True,{'url_param': {}},3,2.0,21,6.0,6.0,...,370.0,True,success,387.0,True,success,225.0,True,nothing extracted,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,yXSA3IXltIEFR810d8h0w9ci,2025-02-26T17:41:38.610307+01:00,2025-02-27 00:53:35.725324+01:00,True,{'url_param': {}},3,0.0,40,8.0,9.0,...,23.0,True,success,58.0,True,success,2192.0,True,success,7.0
6244,ydWKoluEadRNLeowDJTyox64,2025-02-27T08:24:09.358982+01:00,2025-02-27 08:39:50.856565+01:00,True,{'url_param': {}},3,0.0,25,7.0,0.0,...,2.0,True,success,22.0,True,success,124.0,True,success,1.0
6274,ytB953Jkrkke6nmjCP8jIDEd,2025-02-27T00:42:52.265772+01:00,2025-02-27 01:18:48.141486+01:00,True,{'url_param': {}},3,1.0,25,6.0,9.0,...,41.0,True,success,41.0,True,success,391.0,True,nothing extracted,0.0
6275,yu7eCbHwBlEobge28s7A9x6h,2025-02-27T07:18:34.097943+01:00,2025-02-27 07:23:46.795611+01:00,True,{'url_param': {}},3,0.0,26,7.0,0.0,...,0.0,True,success,8.0,True,success,93.0,True,success,1.0


In [11]:
sus_ids = df_failed.participant_id

In [12]:
for id in sus_ids:
    ddp = 

SyntaxError: invalid syntax (591377912.py, line 2)

In [10]:
df_withWH = df_consent[df_consent["AngeseheneVideos_n_datapoints"] > 0]
len(df_withWH)

569

In [11]:

df_usable = df_usable[df_usable["AngeseheneVideos_n_datapoints"] > 0]
df_usable.to_csv('./data/overview/usable_overview.csv', index=False)


Age statistics:
count    320.00
mean      28.37
std        9.29
min       13.00
50%       26.00
max       69.00
Name: Q2_age, dtype: float64


In [13]:
# Use this instead:
pd.concat([
    df_usable.loc[:,['Q1_gender','Q2_age','Q3_education','Q7_polInt-0']].describe(percentiles=[0.5]).round(2), 
    df_usable.loc[:,['Q2_age']].dropna().astype(int).describe(percentiles=[0.5]).round(2)
], axis=1)


,Q1_gender,Q2_age,Q3_education,Q7_polInt-0,Q2_age
count,320.00,320.00,320.00,264.00,320.00
mean,0.48,28.37,5.97,7.33,28.37
std,0.55,9.29,2.22,1.48,9.29
min,0.00,13.00,0.00,2.00,13.00
50%,0.00,26.00,6.00,7.00,26.00
max,3.00,69.00,10.00,9.00,69.00
